In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

# Imports 

In [3]:
import re
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from transformers import DistilBertModel, DistilBertTokenizer, AutoTokenizer

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

In [5]:
import sys
sys.path.append('/lfs/1/sahaana/enrichment/ember/utils')
    
"""from model_utils import MatchedDatasetTriplets, param_header_bert, tokenize_batch   
from models import BatchedTripletSingleBERTModel, BatchedTripletDoubleBERTModel
from model_runner import train_model, eval_model
from knn_utils import FaissKNeighbors, knn_matching_accuracy, find_perfect_recall
from common_utils import get_sorted_files"""

'from model_utils import MatchedDatasetTriplets, param_header_bert, tokenize_batch   \nfrom models import BatchedTripletSingleBERTModel, BatchedTripletDoubleBERTModel\nfrom model_runner import train_model, eval_model\nfrom knn_utils import FaissKNeighbors, knn_matching_accuracy, find_perfect_recall\nfrom common_utils import get_sorted_files'

In [6]:
%load_ext autoreload
%autoreload 2

# Load Data

In [7]:
path = "/lfs/1/sahaana/enrichment/data/SQuAD/"

In [36]:
!ls /lfs/1/sahaana/enrichment/data/SQuAD/

dev_tableA.pkl		    train_tableB_para.pkl
dev_tableB_para.pkl	    train_tableB_para_processed.pkl
dev_tableB_sent.pkl	    train_tableB_sent.pkl
dev-v2.0.json		    train_tableB_sent_processed.pkl
train_tableA.pkl	    train-v2.0.json
train_tableA_processed.pkl


In [10]:
file = "train-v2.0.json"
train = pd.read_json(f"{path}/{file}")
train#,sep='\t', header=None, index_col=0, names=['PID','Passage']

,version,data
0,v2.0,"{'title': 'Beyoncé', 'paragraphs': [{'qas': [{..."
1,v2.0,"{'title': 'Frédéric_Chopin', 'paragraphs': [{'..."
2,v2.0,{'title': 'Sino-Tibetan_relations_during_the_M...
3,v2.0,"{'title': 'IPod', 'paragraphs': [{'qas': [{'qu..."
4,v2.0,{'title': 'The_Legend_of_Zelda:_Twilight_Princ...
...,...,...
437,v2.0,"{'title': 'Infection', 'paragraphs': [{'qas': ..."
438,v2.0,"{'title': 'Hunting', 'paragraphs': [{'qas': [{..."
439,v2.0,"{'title': 'Kathmandu', 'paragraphs': [{'qas': ..."
440,v2.0,"{'title': 'Myocardial_infarction', 'paragraphs..."


In [11]:
file = "dev-v2.0.json"
dev = pd.read_json(f"{path}/{file}")
dev#,sep='\t', header=None, index_col=0, names=['PID','Passage']

,version,data
0,v2.0,"{'title': 'Normans', 'paragraphs': [{'qas': [{..."
1,v2.0,"{'title': 'Computational_complexity_theory', '..."
2,v2.0,"{'title': 'Southern_California', 'paragraphs':..."
3,v2.0,"{'title': 'Sky_(United_Kingdom)', 'paragraphs'..."
4,v2.0,"{'title': 'Victoria_(Australia)', 'paragraphs'..."
5,v2.0,"{'title': 'Huguenot', 'paragraphs': [{'qas': [..."
6,v2.0,"{'title': 'Steam_engine', 'paragraphs': [{'qas..."
7,v2.0,"{'title': 'Oxygen', 'paragraphs': [{'qas': [{'..."
8,v2.0,"{'title': '1973_oil_crisis', 'paragraphs': [{'..."
9,v2.0,"{'title': 'European_Union_law', 'paragraphs': ..."


# Parse Data

In [12]:
train_dataset = {'title': [], 
                 'question': [], 
                 'QID': [], 
                 'paragraph': [],
                 'PID': [],  
                 'sentence': [], 
                 'SID': [],
                 'answer': []}
PID = 0
SID = 0
QID = 0

for block in train['data']:
    for entry in block['paragraphs']:
        sentences = re.split(r'(\.)', entry['context'])
        sentence_lens = np.cumsum([len(i) for i in sentences])
        SIDs = [SID + i for (i,_) in enumerate(sentences)]
        SID += len(sentences)
        for qas in entry['qas']:
            if not qas['is_impossible']:
                for answers in qas['answers']:
                    s_idx = np.searchsorted(sentence_lens, answers['answer_start'])
                    
                    train_dataset['title'].append(block['title'])
                    train_dataset['paragraph'].append(entry['context'])
                    train_dataset['PID'].append(PID)
                    train_dataset['question'].append(qas['question'])
                    train_dataset['QID'].append(QID)
                    train_dataset['answer'].append(answers['text'])
                    train_dataset['sentence'].append(sentences[s_idx])
                    train_dataset['SID'].append(SIDs[s_idx])
                QID += 1
        PID += 1               
        

In [13]:
train_dataset = pd.DataFrame(train_dataset)

In [14]:
train_dataset

,title,question,QID,paragraph,PID,sentence,SID,answer
0,Beyoncé,When did Beyonce start becoming popular?,0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,0,"Born and raised in Houston, Texas, she perfor...",2,in the late 1990s
1,Beyoncé,What areas did Beyonce compete in when she was...,1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,0,"Born and raised in Houston, Texas, she perfor...",2,singing and dancing
2,Beyoncé,When did Beyonce leave Destiny's Child and bec...,2,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,0,Their hiatus saw the release of Beyoncé's deb...,6,2003
3,Beyoncé,In what city and state did Beyonce grow up?,3,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,0,"Born and raised in Houston, Texas, she perfor...",2,"Houston, Texas"
4,Beyoncé,In which decade did Beyonce become famous?,4,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,0,"Born and raised in Houston, Texas, she perfor...",2,late 1990s
...,...,...,...,...,...,...,...,...
86816,Kathmandu,In what US state did Kathmandu first establish...,86816,"Kathmandu Metropolitan City (KMC), in order to...",18979,KMC's first international relationship was es...,228267,Oregon
86817,Kathmandu,What was Yangon previously known as?,86817,"Kathmandu Metropolitan City (KMC), in order to...",18979,This activity has been further enhanced by es...,228269,Rangoon
86818,Kathmandu,With what Belorussian city does Kathmandu have...,86818,"Kathmandu Metropolitan City (KMC), in order to...",18979,This activity has been further enhanced by es...,228269,Minsk
86819,Kathmandu,In what year did Kathmandu create its initial ...,86819,"Kathmandu Metropolitan City (KMC), in order to...",18979,KMC's first international relationship was es...,228267,1975


In [15]:
dev_dataset = {'title': [], 
              'question': [], 
              'QID': [], 
              'paragraph': [],
              'PID': [],  
              'sentence': [], 
              'SID': [],
              'answer': []}

for block in dev['data']:
    for entry in block['paragraphs']:
        sentences = re.split(r'(\.)', entry['context'])
        sentence_lens = np.cumsum([len(i) for i in sentences])
        SIDs = [SID + i for (i,_) in enumerate(sentences)]
        SID += len(sentences)
        for qas in entry['qas']:
            if not qas['is_impossible']:
                for answers in qas['answers']:
                    s_idx = np.searchsorted(sentence_lens, answers['answer_start'])
                    
                    dev_dataset['title'].append(block['title'])
                    dev_dataset['paragraph'].append(entry['context'])
                    dev_dataset['PID'].append(PID)
                    dev_dataset['question'].append(qas['question'])
                    dev_dataset['QID'].append(QID)
                    dev_dataset['answer'].append(answers['text'])
                    dev_dataset['sentence'].append(sentences[s_idx])
                    dev_dataset['SID'].append(SIDs[s_idx])
                QID += 1
        PID += 1               
        

In [17]:
dev_dataset

,title,question,QID,paragraph,PID,sentence,SID,answer
0,Normans,In what country is Normandy located?,86821,The Normans (Norman: Nourmands; French: Norman...,19035,The Normans (Norman: Nourmands; French: Norman...,228945,France
1,Normans,In what country is Normandy located?,86821,The Normans (Norman: Nourmands; French: Norman...,19035,The Normans (Norman: Nourmands; French: Norman...,228945,France
2,Normans,In what country is Normandy located?,86821,The Normans (Norman: Nourmands; French: Norman...,19035,The Normans (Norman: Nourmands; French: Norman...,228945,France
3,Normans,In what country is Normandy located?,86821,The Normans (Norman: Nourmands; French: Norman...,19035,The Normans (Norman: Nourmands; French: Norman...,228945,France
4,Normans,When were the Normans in Normandy?,86822,The Normans (Norman: Nourmands; French: Norman...,19035,The Normans (Norman: Nourmands; French: Norman...,228945,10th and 11th centuries
...,...,...,...,...,...,...,...,...
20297,Force,What is the seldom used force unit equal to on...,92748,"The pound-force has a metric counterpart, less...",20238,Other arcane units of force include the sthèn...,244304,sthène
20298,Force,What is the seldom used force unit equal to on...,92748,"The pound-force has a metric counterpart, less...",20238,Other arcane units of force include the sthèn...,244304,sthène
20299,Force,What is the seldom used force unit equal to on...,92748,"The pound-force has a metric counterpart, less...",20238,Other arcane units of force include the sthèn...,244304,sthène
20300,Force,What is the seldom used force unit equal to on...,92748,"The pound-force has a metric counterpart, less...",20238,Other arcane units of force include the sthèn...,244304,sthène


# Split/Normalize Data

In [20]:
train_paragraphs = train_dataset[['PID', 'paragraph']].set_index('PID').drop_duplicates()
dev_paragraphs = dev_dataset[['PID', 'paragraph']].set_index('PID').drop_duplicates()

In [21]:
train_sentences = train_dataset[['SID', 'sentence']].set_index('SID').drop_duplicates()
dev_sentences = dev_dataset[['SID', 'sentence']].set_index('SID').drop_duplicates()

In [22]:
train_queries = train_dataset[['QID', 'question']].set_index('QID').drop_duplicates()
dev_queries = dev_dataset[['QID', 'question']].set_index('QID').drop_duplicates()

In [201]:
file = "train_tableA.pkl"
train_queries.to_pickle(f"{path}/{file}")

In [208]:
file = "dev_tableA.pkl"
dev_queries.to_pickle(f"{path}/{file}")

In [203]:
file = "train_tableB_sent.pkl"
train_sentences.to_pickle(f"{path}/{file}")

In [204]:
file = "dev_tableB_sent.pkl"
dev_sentences.to_pickle(f"{path}/{file}")

In [212]:
file = "train_tableB_para.pkl"
train_paragraphs.to_pickle(f"{path}/{file}")

In [206]:
file = "dev_tableB_para.pkl"
dev_paragraphs.to_pickle(f"{path}/{file}")

In [37]:
file = "train_all.pkl"
train_dataset.to_pickle(f"{path}/{file}")

In [38]:
file = "dev_all.pkl"
dev_dataset.to_pickle(f"{path}/{file}")

In [140]:
all_SID = set(train_dataset['SID'])
all_title = set(train_dataset['title'])
all_QID = set(train_dataset['QID'])
all_PID = set(train_dataset['PID'])

In [281]:
QID_in_train = set(train_queries.index)
QID_in_dev = set(dev_queries.index)

In [296]:
SID_in_train = set(train_sentences.index)
SID_in_dev = set(dev_sentences.index)

In [297]:
PID_in_train = set(train_paragraphs.index)
PID_in_dev = set(dev_paragraphs.index)

# Compute Labels for Train

In [366]:
sentences_per_para_train = pd.DataFrame(train_dataset[['PID',
                                                       'SID']].groupby('PID')['SID'].apply(lambda x: set([i for i in set(x) if i in SID_in_train])))
sentences_to_para_train = train_dataset[['PID', 
                                         'SID']].drop_duplicates().set_index('SID')
para_per_topic_train = pd.DataFrame(train_dataset[['title', 
                                                   'PID']].groupby('title')['PID'].apply(lambda x: set([i for i in set(x) if i in PID_in_train])))
para_to_topic_train = pd.DataFrame(train_dataset[['title', 
                                                  'PID']].drop_duplicates().set_index('PID'))

In [367]:
train_sentence_labels = train_dataset[['QID', 'SID']]#.set_index('QID')

In [368]:
train_sent_labels = {'QID_a': [], 'SID_p': [], 'SID_n': []}
for _, record in train_sentence_labels.iterrows():
    if record.QID in QID_in_train and record.SID in SID_in_train:
        num_samples = 3
        negatives = set()

        # First get the set of sentences that come from the same paragraph
        paragraph = sentences_to_para_train.loc[record.SID].PID # paragraph index
        sentences = sentences_per_para_train.loc[paragraph].SID.copy() # other sentences in para
        sentences.remove(record.SID) 
        if len(sentences) > 0:
            negatives.add(np.random.choice(list(sentences)))

        # Then get paragraphs that come from the same title/topic
        topic = para_to_topic_train.loc[paragraph].title # title 
        paragraphs = para_per_topic_train.loc[topic].PID.copy() # other paragraphs in this topic
        paragraphs.remove(paragraph)
        if len(paragraphs) > 0:
            sample_para = np.random.choice(list(paragraphs))
            sample_sent = sentences_per_para_train.loc[sample_para].SID.copy()
            if len(sample_sent) > 0:
                negatives.add(np.random.choice(list(sample_sent)))

        # Then get a rando
        while len(negatives) < num_samples:
            negatives.add(np.random.choice(list(SID_in_train)))

        for n in negatives:
            train_sent_labels['QID_a'].append(record.QID)
            train_sent_labels['SID_p'].append(record.SID)
            train_sent_labels['SID_n'].append(n)

In [369]:
file = "train_sent_triplets.pkl"
pd.DataFrame(train_sent_labels).to_pickle(f"{path}/{file}")

In [370]:
file = "train_sent_triplets.pkl"
pd.read_pickle(f"{path}/{file}")

,QID_a,SID_p,SID_n
0,0,2,113322
1,0,2,218
2,0,2,6
3,1,2,71709
4,1,2,6
...,...,...,...
259999,86819,228267,113676
260000,86819,228267,228269
260001,86820,228265,228269
260002,86820,228265,93838


In [371]:
train_paragraph_labels = train_dataset[['QID', 'PID']]#.set_index('QID')
dev_paragraph_labels = dev_dataset[['QID', 'PID']]#.set_index('QID')

In [372]:
train_para_labels = {'QID_a': [], 'PID_p': [], 'PID_n': []}
for _, record in train_paragraph_labels.iterrows():
    if record.QID in QID_in_train and record.PID in PID_in_train:
        num_samples = 3
        negatives = set()

        # First get the paragraphs that come from the same title/topic
        topic = para_to_topic_train.loc[record.PID].title # title
        paragraphs = para_per_topic_train.loc[topic].PID.copy() # other paragraphs in this topic
        paragraphs.remove(record.PID)
        if len(paragraphs) > 0:
            negatives.add(np.random.choice(list(paragraphs)))

        # Then get some randos
        while len(negatives) < num_samples:
            negatives.add(np.random.choice(list(PID_in_train)))

        for n in negatives:
            train_para_labels['QID_a'].append(record.QID)
            train_para_labels['PID_p'].append(record.PID)
            train_para_labels['PID_n'].append(n)

In [373]:
file = "train_para_triplets.pkl"
pd.DataFrame(train_para_labels).to_pickle(f"{path}/{file}")

In [374]:
file = "train_para_triplets.pkl"
pd.read_pickle(f"{path}/{file}")

,QID_a,PID_p,PID_n
0,0,0,9
1,0,0,14692
2,0,0,6701
3,1,0,58
4,1,0,15021
...,...,...,...
260248,86819,18979,18202
260249,86819,18979,14155
260250,86820,18979,2177
260251,86820,18979,5337


In [375]:
train_dataset

,title,question,QID,paragraph,PID,sentence,SID,answer
0,Beyoncé,When did Beyonce start becoming popular?,0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,0,"Born and raised in Houston, Texas, she perfor...",2,in the late 1990s
1,Beyoncé,What areas did Beyonce compete in when she was...,1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,0,"Born and raised in Houston, Texas, she perfor...",2,singing and dancing
2,Beyoncé,When did Beyonce leave Destiny's Child and bec...,2,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,0,Their hiatus saw the release of Beyoncé's deb...,6,2003
3,Beyoncé,In what city and state did Beyonce grow up?,3,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,0,"Born and raised in Houston, Texas, she perfor...",2,"Houston, Texas"
4,Beyoncé,In which decade did Beyonce become famous?,4,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,0,"Born and raised in Houston, Texas, she perfor...",2,late 1990s
...,...,...,...,...,...,...,...,...
86816,Kathmandu,In what US state did Kathmandu first establish...,86816,"Kathmandu Metropolitan City (KMC), in order to...",18979,KMC's first international relationship was es...,228267,Oregon
86817,Kathmandu,What was Yangon previously known as?,86817,"Kathmandu Metropolitan City (KMC), in order to...",18979,This activity has been further enhanced by es...,228269,Rangoon
86818,Kathmandu,With what Belorussian city does Kathmandu have...,86818,"Kathmandu Metropolitan City (KMC), in order to...",18979,This activity has been further enhanced by es...,228269,Minsk
86819,Kathmandu,In what year did Kathmandu create its initial ...,86819,"Kathmandu Metropolitan City (KMC), in order to...",18979,KMC's first international relationship was es...,228267,1975


# Compute Labels for Test

In [380]:
dev_sentence_labels = dev_dataset[['QID', 'SID']]#.set_index('QID')

In [232]:
# note that dev seems to have multiple answers per query? 
dev_sent_labels = pd.DataFrame(dev_sentence_labels.groupby('QID')['SID'].apply(lambda x: list(set(x)))).reset_index()

In [233]:
file = "dev_sent_labels.pkl"
dev_sent_labels.to_pickle(f"{path}/{file}")

In [248]:
# This is always the same, singluar thing
dev_para_labels = pd.DataFrame(dev_paragraph_labels.groupby('QID')['PID'].first()).reset_index()

In [250]:
file = "dev_para_labels.pkl"
dev_para_labels.to_pickle(f"{path}/{file}")

In [381]:
dev_sentence_labels

,QID,SID
0,86821,228945
1,86821,228945
2,86821,228945
3,86821,228945
4,86822,228945
...,...,...
20297,92748,244304
20298,92748,244304
20299,92748,244304
20300,92748,244304
